In [1]:
import json

file_path = "caption_cn.jsonl"

with open(file_path, "r", encoding="utf-8") as f:
    first_line = f.readline()        # 读第一行
    first_data = json.loads(first_line)  # 解析 json

print("第一条数据内容：")
print(first_data)
print("\n字段名：")
print(first_data.keys())


第一条数据内容：
{'id': 0, 'image': 'image_comparison/0001/00000001.jpg', 'conversations': [{'from': 'human', 'value': '<image>\n对呈现的图片提供一个详细的描述。'}, {'from': 'gpt', 'value': '这张图片展示了一系列复杂的三维立体图案，由一系列看似3D立方体组成，排列形成"A," "E," "I," "O," 和 "U"字样。立体效果使得某些方块看起来在其他方块的上方，有些则似乎处于下方，创造了一个视觉错觉效果，使得这些字母有的朝上，有的旋转角度不同。'}]}

字段名：
dict_keys(['id', 'image', 'conversations'])


In [1]:
import json

input_path = "caption_cn.jsonl"
output_path = "caption_cn_first30.json"

results = []

with open(input_path, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        if idx >= 143548:
            break

        item = json.loads(line)

        image = item["image"]

        conversations = []
        for turn in item["conversations"]:
            if turn["from"] == "human":
                conversations.append({
                    "role": "user",
                    "content": turn["value"]
                })
            elif turn["from"] == "gpt":
                conversations.append({
                    "role": "assistant",
                    "content": turn["value"]
                })

        results.append({
            "image": image,
            "conversations": conversations
        })

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"✅ 已转换前 {len(results)} 条数据，保存至 {output_path}")


✅ 已转换前 143548 条数据，保存至 caption_cn_first30.json


In [2]:
import json

def single_to_double(input_json_path, output_json_path):
    # 读取原始单图数据
    with open(input_json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    assert isinstance(data, list), "输入 JSON 必须是 list"

    new_data = []

    # 两两合并
    for i in range(0, len(data), 2):
        if i + 1 >= len(data):
            # 如果是奇数条数据，最后一条可以选择丢弃或单独处理
            break

        item1 = data[i]
        item2 = data[i + 1]

        img1 = item1["image"]
        img2 = item2["image"]

        # 取原 assistant 描述
        desc1 = item1["conversations"][1]["content"]
        desc2 = item2["conversations"][1]["content"]

        merged_item = {
            "images": [img1, img2],
            "conversations": [
                {
                    "role": "user",
                    "content": "<image>\n<image>\n请分别并整体描述这两张图片的内容。"
                },
                {
                    "role": "assistant",
                    "content": (
                        f"第一张图片：{desc1}\n\n"
                        f"第二张图片：{desc2}\n\n"
                        "整体来看，这两张图片在内容和风格上存在明显差异。"
                    )
                }
            ]
        }

        new_data.append(merged_item)

    # 保存新数据
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(new_data, f, ensure_ascii=False, indent=2)

    print(f"转换完成，共生成 {len(new_data)} 条双图数据")


In [3]:
single_to_double(
    input_json_path="caption_cn_first143548.json",
    output_json_path="caption_cn_first143548_double_image_data.json"
)


转换完成，共生成 71774 条双图数据
